In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
# Load the YAML configuration file containing class labels
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

# Extract the list of class labels from the YAML file
labels = data_yaml['names']
print(labels)

['Test Name', 'Value', 'Units', 'Reference Range']


In [3]:
# Load the YOLO model from an ONNX file
yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')

# Set backend to OpenCV and specify that computations will run on the CPU
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
# Load the image to be analyzed
img = cv2.imread('./thyrocare_0_421.jpg')
image = img.copy()  # Create a copy of the original image for drawing predictions
row, col, d = image.shape  # Get the image dimensions

# Resize the image to a square, as YOLO typically expects a square input
max_rc = max(row, col)  # Get the larger dimension (either row or col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)  # Create a square black canvas
input_image[0:row, 0:col] = image  # Place the original image in the square canvas

# Convert the image into a blob (a preprocessed array format used by YOLO)
input_width_yolo = 640  # YOLO input image size (640x640 is a typical input size)
blob = cv2.dnn.blobFromImage(input_image, 1/255, (input_width_yolo, input_width_yolo), swapRB=True, crop=False)

# Set the input for YOLO and run forward pass to get predictions
yolo.setInput(blob)
predictions = yolo.forward()  # Get the raw predictions from YOLO model

In [5]:
print(predictions.shape)  # Print the shape of the predictions (useful for debugging)

(1, 25200, 9)


In [6]:
# Non-Maximum Suppression (NMS) and confidence filtering
# Filter predictions based on confidence threshold (0.4) and class probability threshold (0.25)
detections = predictions[0]
boxes = []          # List to store the bounding box coordinates
confidences = []    # List to store the confidence scores
classes = []        # List to store the class IDs
cropped_images = []    # To store cropped images of detected regions

# Get the width and height of the input image
image_w, image_h = input_image.shape[:2]
x_factor = image_w / input_width_yolo  # Scaling factor for x dimension
y_factor = image_h / input_width_yolo  # Scaling factor for y dimension

# Iterate over each detection to filter out low confidence ones
for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]  # Confidence of detecting an object
    if confidence > 0.4:  # Only consider detections with confidence > 0.4
        class_score = row[5:].max()  # Maximum class probability score
        class_id = row[5:].argmax()  # ID of the class with the highest probability

        if class_score > 0.25:  # Consider class predictions with score > 0.25
            # Extract the bounding box parameters (center x, center y, width, height)
            cx, cy, w, h = row[0:4]

            # Convert to bounding box format (top-left corner and width/height)
            left = int((cx - 0.5 * w) * x_factor)
            top = int((cy - 0.5 * h) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)

            # Append the bounding box, confidence score, and class ID to their respective lists
            box = np.array([left, top, width, height])
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

# Convert the list of bounding boxes and confidences to NumPy arrays for NMS
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# Perform Non-Maximum Suppression to eliminate overlapping boxes
# Keeps the box with the highest confidence and removes others with an IoU overlap > 0.45
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

In [7]:
# Draw the bounding boxes and labels on the image
for ind in index:
    # Extract the bounding box coordinates
    x, y, w, h = boxes_np[ind]
    bb_conf = int(confidences_np[ind] * 100)  # Scale the confidence to percentage
    classes_id = classes[ind]  # Get the class ID for this detection
    class_name = labels[classes_id]  # Get the corresponding class name

    # Create the label text (e.g., 'Test Name: 98%')
    text = f'{class_name}: {bb_conf}%'
    print(text)  # Print the label and confidence score to the console

    # Draw the bounding box on the image (green rectangle)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Draw a filled rectangle for the label background (white box)
    cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 255, 255), -1)

    # Add the label text on top of the rectangle (black text)
    cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)

Reference Range: 88%
Value: 85%
Test Name: 79%
Units: 76%


In [8]:
# Display the original and prediction images in separate windows
cv2.imshow('original', img)
cv2.imshow('yolo_prediction', image)
cv2.waitKey(0)  # Wait for a key press to close the windows
cv2.destroyAllWindows()  # Close all OpenCV windows